In [14]:
import numpy as np
import pandas as pd
import cv2

In [15]:
class pnp():

  def __init__(self, obj_points, img_points, camera_matrix, dist_coeff):
    self.obj_points = obj_points
    self.img_points = img_points
    self.camera_matrix = camera_matrix
    self.dist_coeff = np.zeros((4,1))
  
  def train_model(self):
    (self.success, self.r_vec, self.t_vec) = cv2.solvePnP(objectPoints=self.obj_points,
                                                          imagePoints=self.img_points,
                                                          cameraMatrix=self.camera_matrix,
                                                          distCoeffs=self.dist_coeff)

    self.rotation_matrix = cv2.Rodrigues(self.r_vec)[0]

    self.matrix = np.concatenate((self.rotation_matrix, self.t_vec), axis=1).astype('float64')
    
    return self.success, self.matrix

  def estimate_points(self, points_2d): # points_2d needs to be in the form (u, v, 1) as column vector
    points_3d = np.matmul(np.linalg.pinv(self.matrix).astype('float64'),points_2d.astype('float64'))
    return points_3d

In [16]:
#take input from data
obj_points = pd.DataFrame.to_numpy(pd.read_excel("obj_points.xlsx",header=None)).astype("float64")
img_points = pd.DataFrame.to_numpy(pd.read_excel("img_points.xlsx", header=None)).astype("float64")
dist_coeff = np.zeros((4,1))
camera_matrix = pd.DataFrame.to_numpy(pd.read_excel("camera_matrix.xlsx", header=None)).astype("float64")

In [17]:
#instantiate the model
model = pnp(obj_points, img_points, camera_matrix, dist_coeff)
# camera_matrix = [[focal_length, 0, center_x], [0, focal_length, center_y], [0,0,1]]
model.train_model()

(True,
 array([[ 4.36806573e-01,  7.01371649e-01,  5.63274203e-01,
          4.48482065e+02],
        [-8.99197835e-01,  3.58091708e-01,  2.51423112e-01,
          4.96659135e+02],
        [-2.53627789e-02, -6.16318212e-01,  7.87088680e-01,
          2.33247031e+01]]))

In [18]:
#predicting the points in 3d given 2d points
points_2d = pd.DataFrame.to_numpy(pd.read_excel("points_2d.xlsx", header=None))
points_2d = points_2d.astype('float32')
points_3d = model.estimate_points(points_2d)
print(points_3d[:3,:])

[[-73.72323163]
 [ -7.75990805]
 [-37.42640246]]


In [19]:
print("obj_points is:", obj_points)
print("img_points is:", img_points)
print("points_2d is:", points_2d)

obj_points is: [[   0.    0.    0.]
 [   0. -330.  -65.]
 [-225.  170. -135.]
 [ 225.  170. -135.]
 [-150. -150. -125.]
 [ 150. -150. -125.]]
img_points is: [[359. 391.]
 [399. 561.]
 [337. 297.]
 [513. 301.]
 [345. 465.]
 [453. 469.]]
points_2d is: [[399.]
 [561.]
 [  1.]]
